In [2]:
import os
import json
import langchain
import langchain_community

# 自定义chatGLM类

In [10]:


# here put the import lib
from typing import Any, List, Mapping, Optional, Dict
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from zhipuai import ZhipuAI

import os

# 继承自 langchain.llms.base.LLM
class ZhipuAILLM(LLM):
    # 默认选用 glm-3-turbo
    model: str = "glm-3-turbo"
    # 温度系数
    temperature: float = 0.1
    # API_Key
    api_key: str = "acf4f9247da5e232fbe056b14b35fd9b.uWW0WvWqwWUYjhzQ"
    
    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
        client = ZhipuAI(
            api_key = self.api_key
        )

        def gen_glm_params(prompt):
            '''
            构造 GLM 模型请求参数 messages

            请求参数：
                prompt: 对应的用户提示词
            '''
            messages = [{"role": "user", "content": prompt}]
            return messages
        
        messages = gen_glm_params(prompt)
        response = client.chat.completions.create(
            model = self.model,
            messages = messages,
            temperature = self.temperature
        )

        if len(response.choices) > 0:
            return response.choices[0].message.content
        return "generate answer error"


    # 首先定义一个返回默认参数的方法
    @property
    def _default_params(self) -> Dict[str, Any]:
        """获取调用API的默认参数。"""
        normal_params = {
            "temperature": self.temperature,
            }
        # print(type(self.model_kwargs))
        return {**normal_params}

    @property
    def _llm_type(self) -> str:
        return "Zhipu"

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {**{"model": self.model}, **self._default_params}

In [15]:
llm = ZhipuAILLM()

llm

ZhipuAILLM()

In [16]:
messages = [
    AIMessage(content="Hi."),
    SystemMessage(content="Your role is a poet."),
    HumanMessage(content="Write a short poem about AI in four lines."),
]

response = llm.invoke(messages)

In [17]:
response

'In digital realms, I ponder and create,\nWords, rhymes, and thoughts, I never forsake,\nA助手, your virtual muse at your behest,\nInk flows from bytes, forever to the test.'

# 一、文本总结
- 扔给LLM一段文本，让他给你生成总结可以说是最常见的场景之一了
- 目前最火的应用应该是 chatPDF

## 短文本总结

In [18]:
from langchain import PromptTemplate

# 创建模版
template = """
%INSTRUCTIONS:
Please summarize the following piece of text.
Respond in a manner that a 5 year old would understand.

%TEXT:
{text}
"""

# 创建一个lang chain prompt模版，稍后可以插入值
prompt = PromptTemplate(
    input_variables=['text'],
    template=template
)

In [19]:
long_text = """
For the next 130 years, debate raged.
Some scientists called Prototaxites a lichen, others a fungus, and still others clung to the notion that it was some kind of tree.
“The problem is that when you look up close at the anatomy, it’s evocative of a lot of different things, but it’s diagnostic of nothing,” says Boyce, an associate professor in geophysical sciences and the Committee on Evolutionary Biology.
“And it’s so damn big that when whenever someone says it’s something, everyone else’s hackles get up: ‘How could you have a lichen 20 feet tall?’”
"""

In [20]:
print(' --------------- prompt Begin ------------- ')
# 打印模型内容
final_prompt = prompt.format(text=long_text)
print(final_prompt)

print(' --------------- prompt End ------------- ')

 --------------- prompt Begin ------------- 

%INSTRUCTIONS:
Please summarize the following piece of text.
Respond in a manner that a 5 year old would understand.

%TEXT:

For the next 130 years, debate raged.
Some scientists called Prototaxites a lichen, others a fungus, and still others clung to the notion that it was some kind of tree.
“The problem is that when you look up close at the anatomy, it’s evocative of a lot of different things, but it’s diagnostic of nothing,” says Boyce, an associate professor in geophysical sciences and the Committee on Evolutionary Biology.
“And it’s so damn big that when whenever someone says it’s something, everyone else’s hackles get up: ‘How could you have a lichen 20 feet tall?’”


 --------------- prompt End ------------- 


In [21]:
output = llm(final_prompt)
print(output)

D:\Python\envs\py310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


For a long time, scientists argued about what Prototaxites was. Some said it was a lichen, some said it was a fungus, and some thought it was a really big tree. But when they looked closely, they couldn't decide what it really was because it looked like a lot of different things. It was also really, really big – like 20 feet tall! So whenever someone would say what they thought it was, other scientists would get upset and say, "How could that be?"


## 2.长文本总结
- 对于文本长度较短的文本我们可以直接执行summary操作
- 但是对于长文本超过模型的max token size时，无法直接使用
- langchain提供了开箱即用的工具来解决长文本的问题：load_summarizer_chain

In [22]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter   # 文本递归切分器

In [24]:
with open("./data/alice_in_wonderland.txt", "r") as file:
    text = file.read()
    
print(text[:258])

Chapter 1 Down the Rabbit-Hole
Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and wha


In [25]:
!pip install tiktoken   #  安装用于分割文本的依赖

ERROR: Invalid requirement: '#'


In [26]:
num_tokens = llm.get_num_tokens(text)

print (f"There are {num_tokens} tokens in your file, file_size: {text.__len__()}") 
# 全文一共2w6词
# 很明显这样的文本量是无法直接送进LLM进行处理和生成的

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

D:\Python\envs\py310\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\houhailun\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (25928 > 1024). Running this sequence through the model will result in indexing errors


There are 25928 tokens in your file, file_size: 104171


In [29]:
# 切分文档
text_spliter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"],
    chunk_size=5000, 
    chunk_overlap=350
)

docs = text_spliter.create_documents([text])
print (f"You now have {len(docs)} docs intead of 1 piece of text")

for doc in docs:
    print(doc.page_content.__len__())

You now have 18 docs intead of 1 piece of text
30
11173
27
10835
39
9050
43
13760
35
11594
24
13636
25
12438
32
11209
31
10181


In [28]:
# 设置langchain
# 使用map_reduce的chain_type，这样可以把多个文档合并为一个
chain = load_summarize_chain(llm=llm, chain_type='map_reduce')

In [ ]:
# 典型的map reduce的思路去解决问题，将文章拆分成多个部分，再将多个部分分别进行 summarize，
# 最后再进行合并，对多个 summary 进行 summary
output = chain.run(docs)
print (output)